# Chainerチュートリアル

Encoder-Decoderモデルへの前準備としてのChainerイントロダクション。

公式のIntroduction to Chainerに目を通した方を対象にしています。（Chainer ver2）  
https://docs.chainer.org/en/stable/tutorial/basic.html

## Chainerの特徴

ChainerはDefine by Runという思想の深層学習フレームワークです。  
Define and Runなフレームワークと異なり、可変長系列の処理などが、より直感的書ける、またよりpythonicに書けるという利点があります。  

Chainerは計算過程を保持しつつnumpy.arrayのように扱えるデータ型`Variable`をサポートしており、保持された計算過程を通して、自動的にbackpropagationが行われるようになっています。  
そのあたりは公式のIntroductionが詳しいので、そちらを参照してください。  
https://docs.chainer.org/en/stable/tutorial/basic.html  


Chainerにおいては、クラスを用いてモデル記述を行います。  
属性にニューラルネットワークのレイヤー、メソッドにそれらをいかに組み合わせて計算するかを記述します。

In [1]:
class Model:
    
    def __init__(self): # 属性にどのようなレイヤーを用いるかを記述
        self.layer1 = ...
        self.layer2 = ...
        
    def __call__(self, x): # __call__メソッドにどのようにレイヤーを用いるかを記述
        h1 = self.layer1(x)
        h2 = self.layer2(h1)
        ...
        ...
        
        return output

Chainというスーパークラスを継承することで、モデルを保存したり、CPUとGPUの切り替わりにうまく対応したりすることができます。

In [ ]:
class Model(Chain):
    def __init__(self):
        super(Model, self).__init__() # Chainを継承
        with self.init_scope():
            self.layer1 = ...
            self.layer2 = ...
    
    def __call__(self, x):
        h1 = self.layer1(x)
        h2 = self.layer2(h1)
        ...
        ...
        
        return output

これがChainerでモデルを記述する際の基本骨子となります。

## Chainerのレイヤー

Chainerでは様々なニューラルネットワークアーキテクチャが予め用意されています。  
公式のReferenceを見ると、代表的なものはほぼ網羅されているのがわかります。  
https://docs.chainer.org/en/stable/reference/links.html

以下では、Encoder-Decoderモデルを記述する上で必要なレイヤーについて解説していきます。

その前にまずはライブラリをインポートしておきましょう。

In [1]:
import numpy as np
import chainer
from chainer import cuda, Function, gradient_check, report, training, utils, Variable
from chainer import datasets, iterators, optimizers, serializers
from chainer import Link, Chain, ChainList
import chainer.functions as F
import chainer.links as L
from chainer.training import extensions

公式のチュートリアルにあるものの丸写しです。  
使わないものもあります。

### chainer.links.Linear

シンプルな線形変換の関数です。

\begin{equation}
h = Wx + b
\end{equation}

よく見るこれです。  
入力次元と出力次元を指定して使います。

試しに使ってみましょう。

In [2]:
# 入力；50次元 -> 出力；10次元
linear = L.Linear(in_size=50, out_size=10)

In [3]:
# 20個の50次元の疑似データ
x = np.random.normal(size=(20, 50)).astype('f')
# dtypeをfloat32にするのを忘れないように。

In [4]:
x.shape

(20, 50)

In [5]:
linear(x).shape

(20, 10)

なお、Linearの場合、in_sizeを指定しなくてもかまいません。  
in_sizeを指定しないときは、入力ベクトルの次元に自動的に合わせてくれます。

In [6]:
linear = L.Linear(10)
linear(x).shape

(20, 10)

基本的にこの関数と、好きな活性化関数を用いれば、多層パーセプトロンを書くことができます。

In [7]:
class MLP(Chain):
    # 二つの中間層(100 -> 50)を持つ4クラス分類の多層パーセプトロン
    def __init__(self):
        super(MLP, self).__init__()
        with self.init_scope():
            self.l1 = L.Linear(100)
            self.l2 = L.Linear(50)
            self.l3 = L.Linear(4)
    
    def __call__(self, x):
        
        # 活性化関数はtanh
        h1 = F.tanh(self.l1(x))
        h2 = F.tanh(self.l2(h1))
        
        # 出力。これをsoftmaxなどにかければよい。
        y = self.l3(h2)
        return y

In [8]:
mlp = MLP()

In [9]:
mlp(x).shape

(20, 4)

### chainer.links.EmbedID

単語id(int)の系列を、idに対応するembeddingの系列に変換してくれるレイヤーです。  
もちろん、単語でなくとも構いません。  
自然言語処理ではよく使います。  

これもin_sizeとout_sizeを指定して使います。  
in_sizeはボキャブラリー数、out_sizeはembeddingの次元数です。

In [10]:
# 1000ボキャブラリーで50次元のembedding
embed_id = L.EmbedID(in_size=1000, out_size=50)

In [11]:
# 20個で、長さ40の系列の疑似データ
xs = np.random.randint(0, 1000, (20, 40)).astype('i')
# dtypeをint32にするのを忘れないように。

In [12]:
embed_id(xs).shape

(20, 40, 50)

このshapeは(バッチサイズ * 系列の長さ * emmbedingの次元)となっています。  
これが、LSTMなどのRNNの入力となっていきます。

なお、initialWという項を指定することで、word2vecなどで獲得したembeddingを初期値することが可能です。

### chainer.links.NStepLSTM

可変長系列に対応したLSTMです。  
n_layers(レイヤー数)、in_size(入力ベクトル次元)、out_size(隠れ状態次元)、dropout(ドロップアウト率)を指定して使います。  
dropoutは縦方向にかかります。


特徴的なのは、arrayではなく、arrayのlistが入力となるところです。  
arrayのlistを受け取ることができるので、可変長の系列を扱うことができます。

In [13]:
ns_lstm = L.NStepLSTM(n_layers=2, in_size=50, out_size=100, dropout=0)

In [14]:
# 0~999ボキャブラリーのidを１〜１５系列までランダムに並べた２０サンプルの疑似データ
xs = [np.random.randint(0, 1000, size=np.random.randint(1, 15, 1)).astype('i')
      for i in range(20)]

In [15]:
xs

[array([722, 457, 925, 479, 393,  86, 890, 826, 588,  80, 169, 295], dtype=int32),
 array([777, 633, 693, 165, 121, 219, 424, 957], dtype=int32),
 array([356, 583, 715, 923, 633, 668, 647, 877, 352], dtype=int32),
 array([682, 566, 787, 310, 394,  19], dtype=int32),
 array([908, 614, 555, 392, 640, 424, 856, 306, 792, 762, 693], dtype=int32),
 array([147, 694, 925, 560, 430, 144, 699, 362, 806, 524, 543], dtype=int32),
 array([111, 169, 145, 304], dtype=int32),
 array([778, 962, 949, 319, 867, 475, 518, 915,  87], dtype=int32),
 array([773, 788, 557, 605, 805, 307, 748, 210, 185, 225, 647], dtype=int32),
 array([662,   9, 252, 401], dtype=int32),
 array([377, 173, 318, 181, 627, 344, 831, 530,   0, 474, 519], dtype=int32),
 array([689, 898, 697, 601, 711, 806, 299, 571, 166, 523], dtype=int32),
 array([513, 989, 893, 759], dtype=int32),
 array([545, 596], dtype=int32),
 array([ 72, 856, 617, 601, 536, 344], dtype=int32),
 array([714, 903, 370, 274, 298, 303,  73, 749], dtype=int32),
 a

In [16]:
embed = [embed_id(x) for x in xs]
for i in embed:
    print(i.shape)

(12, 50)
(8, 50)
(9, 50)
(6, 50)
(11, 50)
(11, 50)
(4, 50)
(9, 50)
(11, 50)
(4, 50)
(11, 50)
(10, 50)
(4, 50)
(2, 50)
(6, 50)
(8, 50)
(13, 50)
(14, 50)
(6, 50)
(11, 50)


callする際の引数は、hx, cx, xsの３つです。それぞれ、

- hx: 最初の隠れ状態
- cy: 最初のcellの状態
- xs: 入力（embedding）の系列

となります。
hx, cyは`None`を指定すると、自動的にzeroベクトルで埋めてくれます。

In [17]:
hy, cy, ys = ns_lstm(hx=None, cx=None, xs=embed)

hyは最後のタイムステップの隠れ状態。  
cyは最後のタイムステップのcellの状態。  
ysは各タイムステップの隠れ状態です。

In [18]:
hy.shape

(2, 20, 100)

hyは(n_layers, batch_size, outsize)のshapeになっています。  
今回は二層のlstmなので、hy[1]には二層目の(batch_size, outsize)のベクトルが入っています。

In [19]:
cy.shape

(2, 20, 100)

cyも同様です。

In [20]:
type(ys)

list

ysはbatch_sizeの長さを持つリストです。

In [21]:
ys[0].shape

(12, 100)

各要素は(n_time_step, outsize)のshapeのVariableで、各time_stepのhidden vectorが入っています。

これらのレイヤーを利用して、Encoder-Decoderモデルを実装することが可能です。